# py0xcluster

py0xCluster is a package dedicated to perform exploratory data analysis and machine learning tasks on DEX activity (Decentralized Exhanges) and web3 data.

This is for now an educational project for myself with the aim of performing fun data-science projects around blockchain data gathered through the Graph Network (https://thegraph.com/)

## Target objectives

- Establish meaningful grouping of address by clustering DEX traders and LP
    - Feature Extraction:
        - TBD but based on mint/swap/burn data from messari subgraphs entities
        - with or without balances at swap time (web3py fetch balance at block)
        - EOA vs Contracts
    - Dimensionality reduction:
        - UMAP / tSNE or PCA / ICA
    - Clustering:
        - DBSCAN
        - silhouette evaluation
    - Visualization:
        - scatter plot with color-coded returns? (TBD)

## Secondary objective: identify which group has the most profitable activity

- Triggered Average of price by swap in/out by group of addresses

- Predict future returns based on the activity of previously clustered groups of addresses

## ML overall approach:

- Decide whether adopting time-series vs tabular approach (preference for the first one)
- Compute time-series based on extracted features and certain kernels / windowing
- Begin by classification approach of expected future (down-bad / neurtral / up-strong)
- Extend to regression

## Random list of potential features:

### Accounts

### Relative to a pool:
- z-scored (clarify how) difference of price 24h? after swap -> could be target independant variable

- nb of events (z-scored to other addresses on same pool)
- average swap size (z-scored/pool)
- average deposit size (z-scored/pool)
- average withdraw size (z-score/pool)

### Account only

- Total nb of positions: swapCount, depositCount, withdrawCount
- ratio? of nb of: swaps / (deposits + withdraws)

### Account - Position
- nb of (liquid) pools interacted with
- % of events (likely swaps) happening in the same block (possibly identical to MEV bots?)
- % of Limit order on uni-v3 (one deposit amout = 0)

### Account - Web3

- is contract?
- Normalized balance (compared to other users) at time of events

# Roadmap:

## Easy / To implement first

### Aggregation / Feature computation

- Aggregate unique addresses
- Implement Account-only query
- First Web3 requests (is_contract / ETH balance)

### First plots

- First features distribution
- PCA/ICA -> t-SNE

## Next, not immediate priority

- Pool clustering / identify easy-best features

### Data Management

- Store/Retrieve to/from SQLite?
- Consider parquet / feather / hdf5

## Secondary, nice to do

### Package

- Update and test requirements / setup
- Document classes and methods with nicely formatted docstrings to future build of the doc

### Performance

- Evaluate performance, profiling, and try improving inefficient / slow bits

### Imports

In [77]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from py0xcluster.utils.web3_utils import *
from py0xcluster.utils.query_utils import *
from py0xcluster.main_classes.pools import *
from py0xcluster.main_classes.pool_events import *

from py0xcluster.features.swaps_feature_extractors import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Gathering data about most-active pools

In [4]:
uni3pools_selector = PoolSelector(
    subgraph_url = 'https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum',
    min_daily_volume_USD = 500000,
    min_TVL = 500000, # Not implemented. consider removing
    start_date = (2022,12,31), 
    end_date = (2023,1,17),
    days_batch_size = 20)

uni3_pools = uni3pools_selector.create_pool_selection(stables='exclude', verbose=True)
uni3_pools.pools_df.head(5)

Queriying from 2022-12-31 00:00:00 to 2023-01-17 00:00:00
1614 lquidity pools snapshots retrieved
330 stable pools snapshots (over 1614) have been removed
174 illiquid pools snapshots (over 1284) have been removed 
67 pools were selected


,pool.name,dailyVolumeUSD,pool.totalValueLockedUSD,token0.lastPriceUSD,token1.lastPriceUSD,pool.protocol.name,pool.protocol.network,pool.id,token0.symbol,token0.decimals,token1.symbol,token1.decimals
0,Uniswap V3 USD Coin/Wrapped Ether 0.05%,2.131306e+08,1.668709e+08,1.000000,1610.914219,Uniswap V3,MAINNET,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,6,WETH,18
1,Uniswap V3 USD Coin/Wrapped Ether 0.01%,4.207116e+07,1.752612e+07,1.000000,1610.914219,Uniswap V3,MAINNET,0xe0554a476a092703abdb3ef35c80e0d76d32939f,USDC,6,WETH,18
2,Uniswap V3 Wrapped Ether/Tether USD 0.05%,3.227337e+07,2.446735e+07,1610.914219,1.000000,Uniswap V3,MAINNET,0x11b815efb8f581194ae79006d24e0d814b7697f6,WETH,18,USDT,6
3,Uniswap V3 Wrapped BTC/Wrapped Ether 0.05%,1.582161e+07,1.372428e+08,22344.217785,1610.914219,Uniswap V3,MAINNET,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,WBTC,8,WETH,18
4,Uniswap V3 Lido DAO Token/Wrapped Ether 0.3%,1.353137e+07,1.263815e+07,2.158034,1610.914219,Uniswap V3,MAINNET,0xa3f558aebaecaf0e11ca4b2199cc5ed341edfd74,LDO,18,WETH,18


## Extracting all events from these pools

### Steps

- Data query:
    - Perform query for each pool, batch by days to accomodate response limit
    - Alternatively the query can be done on multiple pools. Ideally, the size of the batch should be proportional to the volume, but quite arbitrary to implement.
    - Consider whether do swaps / mints / burns separately or jointly
    - Loop and aggregate over days / pools (or batch of pools)


In [17]:
uni3_pools.pools_df.head(3)

,pool.name,dailyVolumeUSD,pool.totalValueLockedUSD,token0.lastPriceUSD,token1.lastPriceUSD,pool.protocol.name,pool.protocol.network,pool.id,token0.symbol,token0.decimals,token1.symbol,token1.decimals
0,Uniswap V3 USD Coin/Wrapped Ether 0.05%,2.131306e+08,1.668709e+08,1.000000,1610.914219,Uniswap V3,MAINNET,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,6,WETH,18
1,Uniswap V3 USD Coin/Wrapped Ether 0.01%,4.207116e+07,1.752612e+07,1.000000,1610.914219,Uniswap V3,MAINNET,0xe0554a476a092703abdb3ef35c80e0d76d32939f,USDC,6,WETH,18
2,Uniswap V3 Wrapped Ether/Tether USD 0.05%,3.227337e+07,2.446735e+07,1610.914219,1.000000,Uniswap V3,MAINNET,0x11b815efb8f581194ae79006d24e0d814b7697f6,WETH,18,USDT,6


In [84]:
uni3_events_getter = PoolEventGetter(
    subgraph_url = 'https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum',
    pools_data = uni3_pools,
    pool_ids = ['0xe0554a476a092703abdb3ef35c80e0d76d32939f','0x11b815efb8f581194ae79006d24e0d814b7697f6'],
    start_date = (2023,1,14), 
    end_date = (2023,1,17),
    days_batch_size = 1
    )

# Get swaps, deposits, and withdraw from a pool.
uni3_events = uni3_events_getter.get_events(verbose=True)


pool: 0xe0554a476a092703abdb3ef35c80e0d76d32939f
Queriying from 2023-01-14 00:00:00 to 2023-01-15 00:00:00
Queriying from 2023-01-15 00:00:00 to 2023-01-16 00:00:00
Queriying from 2023-01-16 00:00:00 to 2023-01-17 00:00:00
pool: 0x11b815efb8f581194ae79006d24e0d814b7697f6
Queriying from 2023-01-14 00:00:00 to 2023-01-15 00:00:00
Queriying from 2023-01-15 00:00:00 to 2023-01-16 00:00:00
Queriying from 2023-01-16 00:00:00 to 2023-01-17 00:00:00


In [85]:
uni3_events.withdraws

,amountUSD,from,to,timestamp,blockNumber,id,InputTokenAmount0,InputTokenAmount1,pool.id
0,466997.086479,0xe0554a476a092703abdb3ef35c80e0d76d32939f,0xc36442b4a4522e871399cd717abdd847ab11fe88,2023-01-14 03:02:35,16402225,0x0415bf0c826fcee6c51e8b7fa3bdc18538afe876658a...,4.664357e+11,3.606339e+17,0xe0554a476a092703abdb3ef35c80e0d76d32939f
1,7388.733176,0xe0554a476a092703abdb3ef35c80e0d76d32939f,0xc36442b4a4522e871399cd717abdd847ab11fe88,2023-01-14 21:37:47,16407780,0x09d412a190ea67a1d24e54133772b6bde147bc1631b6...,7.352700e+09,2.348690e+16,0xe0554a476a092703abdb3ef35c80e0d76d32939f
2,908637.782495,0xe0554a476a092703abdb3ef35c80e0d76d32939f,0xc36442b4a4522e871399cd717abdd847ab11fe88,2023-01-14 00:45:59,16401546,0x6d220b1f7b178034868fde083e43851f35d46608f2c2...,9.072061e+11,9.240534e+17,0xe0554a476a092703abdb3ef35c80e0d76d32939f
3,185901.147909,0xe0554a476a092703abdb3ef35c80e0d76d32939f,0xc36442b4a4522e871399cd717abdd847ab11fe88,2023-01-14 08:17:59,16403798,0xa41e0cf31a457b71abbf6c148b8f09f3f151a37c456c...,1.850127e+11,5.716702e+17,0xe0554a476a092703abdb3ef35c80e0d76d32939f
4,55328.388799,0xe0554a476a092703abdb3ef35c80e0d76d32939f,0xc36442b4a4522e871399cd717abdd847ab11fe88,2023-01-14 01:45:23,16401839,0xb6b9cbb627fc736b4e93063c90ee376251c5828fe0e1...,5.517477e+10,9.923714e+16,0xe0554a476a092703abdb3ef35c80e0d76d32939f
...,...,...,...,...,...,...,...,...,...
125,13345.676420,0x11b815efb8f581194ae79006d24e0d814b7697f6,0xc36442b4a4522e871399cd717abdd847ab11fe88,2023-01-16 15:17:23,16420192,0x6c03b477ccbbe470424b5b009e28faf6f41d9aab35a1...,4.295812e+18,6.773294e+09,0x11b815efb8f581194ae79006d24e0d814b7697f6
126,47339.134063,0x11b815efb8f581194ae79006d24e0d814b7697f6,0xc36442b4a4522e871399cd717abdd847ab11fe88,2023-01-16 02:58:35,16416520,0x9fb8222b59537161b43a87410e7d064cc80388c87cb5...,1.681056e+19,2.082476e+10,0x11b815efb8f581194ae79006d24e0d814b7697f6
127,609.481224,0x11b815efb8f581194ae79006d24e0d814b7697f6,0xc36442b4a4522e871399cd717abdd847ab11fe88,2023-01-16 21:58:35,16422188,0xbec0eb07e6976897f543a606dee4af02d8104da69216...,1.856410e+17,3.163118e+08,0x11b815efb8f581194ae79006d24e0d814b7697f6
128,90942.230113,0x11b815efb8f581194ae79006d24e0d814b7697f6,0xc36442b4a4522e871399cd717abdd847ab11fe88,2023-01-16 01:25:23,16416058,0xc91d9cbd46982530a70bba7094519d9ab795a40f41fe...,2.740925e+19,4.806498e+10,0x11b815efb8f581194ae79006d24e0d814b7697f6


In [78]:
uni3_events.deposits.dtypes


amountUSD                          float64
from                                string
to                                  string
timestamp                   datetime64[ns]
blockNumber                        float64
id                                  string
InputTokenAmount0                  float64
InputTokenAmount1                  float64
pool.id                           category
amountUSD.zscore                   float64
InputTokenAmount0.zscore           float64
InputTokenAmount1.zscore           float64
is_limit_order                        bool
0                                  float64
sameFromTo                         float64
dtype: object

In [86]:
# Features for swaps

uni3_events.swaps = compute_slippage(uni3_events.swaps)

uni3_events.swaps = uni3_events.swaps.round(2)

uni3_events.swaps = compute_zscore(
    uni3_events.swaps, 
    columns= ['amountInUSD', 'amountOutUSD', 'slippage'],
    group_by= 'pool.id')

uni3_events.swaps = same_from_to(uni3_events.swaps, to_float=True)

addresses_swaps_features = aggregate_features(uni3_events.swaps, method = 'mean',
    columns = ['amountInUSD.zscore', 'amountOutUSD.zscore',\
            'slippage', 'slippage.zscore', 'sameFromTo'],
            group_by = 'from')


# Features from deposits
uni3_events.deposits = compute_zscore(
    uni3_events.deposits, 
    columns=['amountUSD', 'InputTokenAmount0', 'InputTokenAmount1'], 
    group_by= 'pool.id')

uni3_events.deposits = same_from_to(uni3_events.deposits, to_float=True)

uni3_events.deposits = find_limit_orders(uni3_events.deposits)

addresses_deposits_features = aggregate_features(uni3_events.deposits, method = 'mean',
    columns = ['amountUSD.zscore', 'sameFromTo', 'is_limit_order'],
            group_by = 'from')

# Features from withdraws
uni3_events.withdraws = compute_zscore(
    uni3_events.withdraws, 
    columns=['amountUSD', 'InputTokenAmount0', 'InputTokenAmount1'], 
    group_by= 'pool.id')

uni3_events.withdraws = same_from_to(uni3_events.withdraws, to_float=True)

addresses_withdraws_features = aggregate_features(uni3_events.withdraws, method = 'mean',
    columns = ['amountUSD.zscore', 'sameFromTo'],
            group_by = 'from')

In [89]:
addresses_swaps_features

,amountInUSD.zscore.mean,amountOutUSD.zscore.mean,slippage.mean,slippage.zscore.mean,sameFromTo.mean
from,,,,,
0x00000000000124d994209fbb955e0217b5c2eca1,-0.139057,-0.139020,0.21,0.340000,0.0
0x000000000002e33d9a86567c6dfe6d92f6777d1e,-0.210334,-0.210110,-0.14,-0.380611,0.0
0x00000000adc7a93f2ba7752da396bba0b3c45196,0.688586,0.688192,0.07,0.051756,0.0
0x00000000be1973fa757f07c304736debe41c0987,4.171559,4.153020,0.47,0.875311,0.0
0x00000000e82abe086fa8e33007e0de540e367e6a,0.458453,0.457659,0.15,0.216467,0.0
...,...,...,...,...,...
0xffe2031ebfe4062696989cb833bb8a7d8681594b,-0.185221,-0.184853,-0.44,-0.998277,0.0
0xfff40f57bb7b4a26add09cf53be0bf5854206de3,-0.188156,-0.187905,-0.13,-0.360022,0.0
0xfff72197abd1658d70c249710ed0ad0c0f1cc6eb,-0.139537,-0.139559,0.28,0.484122,0.0


In [49]:
uni3_events.deposits['isLimitOrder'] = (uni3_events.deposits['InputTokenAmount0'] == 0) | (uni3_events.deposits['InputTokenAmount1'] == 0)
uni3_events.swaps['isLimitOrder'].astype('float')

uni3_events.deposits['isLimitOrder'].value_counts()

False    1172
Name: is_limit_order, dtype: int64

In [9]:
uni3_events.swaps

,amountInUSD,amountOutUSD,amountIn,amountOut,from,to,timestamp,blockNumber,id,pool.id,slippage,amountInUSD.zscore,amountOutUSD.zscore,amountIn.zscore,amountOut.zscore,slippage.zscore,sameFromTo
0,23.191180,23.112975,2.000000e+16,1.401370e+05,0xf7200f7a475b53446a10421dea95299fa9319a85,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,2022-12-16 23:35:35,16200571,0x030ab62f5725bc08d7b9bdd73a8e762b561c2894214e...,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0.337217,-0.216807,-0.217326,-0.137012,-0.169025,0.013384,False
1,52703.620216,52542.252670,3.150668e+08,4.446492e+19,0x28e38142db0fe24cf6f6c9dbd525b0ab019654e4,0x0087bb802d9c0e343f00510000729031ce00bf27,2022-12-17 08:02:35,16203094,0x04dc2a58b48f035cdf143e6149dbb754ea67d6bcf4b2...,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0.306179,0.032270,0.032568,-0.137193,0.226868,-0.010617,False
2,223169.154160,222441.041107,1.338308e+09,1.890228e+20,0x66c3c5d24bae428a69c72117fd2f84b69f697aed,0xe592427a0aece92de3edee1f18e0157c05861564,2022-12-17 04:33:47,16202056,0x063db8c04f6ea0092980e016e3efc62df6c0f62e5563...,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0.326261,0.838244,0.840969,-0.137193,1.513938,0.004911,False
3,117.619322,117.266320,1.000000e+17,7.052490e+05,0x610052a390e3f9dd5409e30207f25e9872ae1e8a,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,2022-12-17 20:21:11,16206769,0x0b357d2f23e1c0fec4880efaf379b5efbcdf970afe19...,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0.300122,-0.216360,-0.216878,-0.136292,-0.169025,-0.015302,False
4,132380.185391,131961.949528,1.138708e+20,7.988272e+08,0xeeb75213df02f4d3d0c7943a3b76c199bb4b8d99,0x0087bb802d9c0e343f00510000729031ce00bf27,2022-12-16 23:42:11,16200604,0x0b4337fe888335cf6fadefb2fbed4ed3f1f41c774b01...,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0.315935,0.408987,0.410457,0.888870,-0.169025,-0.003073,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11889,17.659435,17.606410,8.359300e+04,1.121773e+16,0x7debc7a189456640196772c874437422c8962879,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,2023-01-16 05:11:59,16417184,0xfa6b3e768771315366408f5e09ff472192a6cd9d3984...,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0.300266,-0.216833,-0.217352,-0.137193,-0.168926,-0.015190,False
11890,49725.924135,49574.578166,2.379437e+08,3.203421e+19,0x1cad683486c7c6f3a21d9a522e5d5cbb02d9c35e,0x0087bb802d9c0e343f00510000729031ce00bf27,2023-01-16 10:16:11,16418689,0xfaf7c5c0e8a873a90523453ab5fdf4149fd66167600d...,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0.304360,0.018191,0.018447,-0.137193,0.116191,-0.012024,False
11891,15027.706913,14982.428293,7.106681e+07,9.547108e+18,0x1c8f6a5f009e051cab9c3851ca2da2c936b2775a,0xa69babef1ca67a37ffaf7a485dfff3382056e78c,2023-01-16 04:21:11,16416930,0xfb59ad31ed4a3ac21a0b1327b332c8a59f5ba78db0fa...,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0.301301,-0.145864,-0.146147,-0.137193,-0.084023,-0.014390,False
11892,975.587742,972.660007,6.162516e+17,4.582884e+06,0x9cbf83727f2f8adc0b5de69181b80d91ee10893d,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,2023-01-16 22:58:35,16422487,0xfceec835cc7d4cf1535324c351b18decc5912b86a5b0...,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0.300100,-0.212304,-0.212808,-0.131640,-0.169025,-0.015319,False


In [10]:
swaps_subset = uni3_events.swaps[0:20]
swaps_subset

,amountInUSD,amountOutUSD,amountIn,amountOut,from,to,timestamp,blockNumber,id,pool.id,slippage,amountInUSD.zscore,amountOutUSD.zscore,amountIn.zscore,amountOut.zscore,slippage.zscore,sameFromTo
0,23.191180,23.112975,2.000000e+16,1.401370e+05,0xf7200f7a475b53446a10421dea95299fa9319a85,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,2022-12-16 23:35:35,16200571,0x030ab62f5725bc08d7b9bdd73a8e762b561c2894214e...,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0.337217,-0.216807,-0.217326,-0.137012,-0.169025,0.013384,False
1,52703.620216,52542.252670,3.150668e+08,4.446492e+19,0x28e38142db0fe24cf6f6c9dbd525b0ab019654e4,0x0087bb802d9c0e343f00510000729031ce00bf27,2022-12-17 08:02:35,16203094,0x04dc2a58b48f035cdf143e6149dbb754ea67d6bcf4b2...,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0.306179,0.032270,0.032568,-0.137193,0.226868,-0.010617,False
2,223169.154160,222441.041107,1.338308e+09,1.890228e+20,0x66c3c5d24bae428a69c72117fd2f84b69f697aed,0xe592427a0aece92de3edee1f18e0157c05861564,2022-12-17 04:33:47,16202056,0x063db8c04f6ea0092980e016e3efc62df6c0f62e5563...,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0.326261,0.838244,0.840969,-0.137193,1.513938,0.004911,False
3,117.619322,117.266320,1.000000e+17,7.052490e+05,0x610052a390e3f9dd5409e30207f25e9872ae1e8a,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,2022-12-17 20:21:11,16206769,0x0b357d2f23e1c0fec4880efaf379b5efbcdf970afe19...,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0.300122,-0.216360,-0.216878,-0.136292,-0.169025,-0.015302,False
4,132380.185391,131961.949528,1.138708e+20,7.988272e+08,0xeeb75213df02f4d3d0c7943a3b76c199bb4b8d99,0x0087bb802d9c0e343f00510000729031ce00bf27,2022-12-16 23:42:11,16200604,0x0b4337fe888335cf6fadefb2fbed4ed3f1f41c774b01...,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0.315935,0.408987,0.410457,0.888870,-0.169025,-0.003073,False
5,7360.939169,7343.001081,6.290472e+18,4.417452e+07,0x065e3dbafcb2c26a978720f9eb4bce6ad9d644a1,0xa69babef1ca67a37ffaf7a485dfff3382056e78c,2022-12-17 02:06:47,16201324,0x0df0e0a8f703fd9ccc2e54ccdff036636a7c7e8fbecf...,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0.243693,-0.182113,-0.182497,-0.080511,-0.169025,-0.058937,False
6,11254.322786,11220.410361,9.574646e+18,6.753119e+07,0x0cac3d1a887206e0f6169222c4504301a8b4b993,0xa69babef1ca67a37ffaf7a485dfff3382056e78c,2022-12-17 17:42:35,16205981,0x0f96593626d195f20e2548976a467001261a47edccb8...,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0.301328,-0.163705,-0.164047,-0.050918,-0.169025,-0.014369,False
7,20082.859628,20022.138284,1.200000e+08,1.694394e+19,0x687f506a6c55497e59819d9b36a3cf39c32bbc97,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,2022-12-17 06:38:35,16202676,0x13a91d8d78937e96ae187ab25706eeddb3131672723d...,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0.302354,-0.121963,-0.122168,-0.137193,-0.018165,-0.013575,False
8,84948.064085,84685.885078,7.260725e+19,5.098242e+08,0xeeb75213df02f4d3d0c7943a3b76c199bb4b8d99,0x0087bb802d9c0e343f00510000729031ce00bf27,2022-12-17 02:07:23,16201327,0x15142e6606641539765e822a6db4ecc21f199b2a6146...,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0.308634,0.184724,0.185511,0.517054,-0.169025,-0.008719,False
9,8806.612203,8780.098968,7.575273e+18,5.313304e+07,0x43e4715ae093a4c86b5ecddb52216c4f879e9672,0xa69babef1ca67a37ffaf7a485dfff3382056e78c,2022-12-16 23:42:23,16200605,0x196b501d8d9a32b639b62e6933128d33142108441c99...,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0.301061,-0.175278,-0.175659,-0.068934,-0.169025,-0.014576,False


In [12]:
# Defining the lambda function that takes one argument
lambda_get_bal = lambda x: get_eth_balance(x['from'], x['blockNumber'])

# Using apply() to apply the lambda function to the DataFrame
swaps_subset['ethBalance'] = swaps_subset.apply(lambda_get_bal, axis=1)


/tmp/ipykernel_263975/1949059704.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  swaps_subset['ethBalance'] = swaps_subset.apply(lambda_get_bal, axis=1)


In [40]:
uni3_events.withdraws

,amountUSD,from,to,timestamp,blockNumber,id,InputTokenAmount0,InputTokenAmount1,pool.id
0,51591.88554535432919427100745190239,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xc36442b4a4522e871399cd717abdd847ab11fe88,1671239483,16201048,0x02b8d8433c55d2e64d1f163af901a708aea0c517fc31...,103491373,29330412625036374338,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed
1,440677.62637029047181132415005696,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xc36442b4a4522e871399cd717abdd847ab11fe88,1671257003,16202502,0x04993b15cd267e5930512c521d0809a802b82568a0b8...,1139754152,212733197924195596181,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed
2,6775.015567503398105400126508848573,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xc36442b4a4522e871399cd717abdd847ab11fe88,1671266423,16203283,0x159c620c7aec86170bce367f83a5990224cf639c9f07...,19225913,3012419148854985678,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed
3,4.11623637648873634370480430774902,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xc36442b4a4522e871399cd717abdd847ab11fe88,1671290579,16205285,0x1653c44ed5b91849799d7066508450e59515d8f11230...,10370,2028110082879374,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed
4,3901.254340630483612745671462433689,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xc36442b4a4522e871399cd717abdd847ab11fe88,1671288215,16205088,0x16b84fa423d400e51df09f51dec0f7e86f15db0eefda...,9226208,2005800020107352950,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed
...,...,...,...,...,...,...,...,...,...
1303,17503.74387603716253563426869217166,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xc36442b4a4522e871399cd717abdd847ab11fe88,1673841071,16416781,0xeb6df16a6ffbab5c885f53ee830edf77337522735fca...,44831289,5108248551388872501,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed
1304,184736.8631641206712988043535701367,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xc36442b4a4522e871399cd717abdd847ab11fe88,1673894315,16421195,0xf6240734e5a79d0502ee562c1bb2bae0f48f8a15935e...,428049493,60644122735795926022,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed
1305,38725.70794051380634878981794372125,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xc36442b4a4522e871399cd717abdd847ab11fe88,1673895587,16421300,0xf7e6637668efaf539e7be548cba3c0873c0e700c96c8...,91555579,12293763452944294864,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed
1306,4565.35996840805837242422359349175,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0xc36442b4a4522e871399cd717abdd847ab11fe88,1673835155,16416291,0xfd4c615e0a66272ba0d900646dc4cad343a4c129bdfc...,10567527,1490023224315948408,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed


In [38]:
from web3 import Web3
from web3.beacon import Beacon

w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
w3.isConnected()
block = web3.eth.get_block('latest')
block['number']
web3.eth.syncing
beacon = Beacon("http://localhost:5052")
beacon.get_syncing()


0

In [ ]:
uni3_events.withdraws.to_pickle('/home/fujiju/Documents/GitHub/py0xcluster/data/20230117_215100_withdraws.pkl')

In [ ]:
deposits = pd.read_pickle('/home/fujiju/Documents/GitHub/py0xcluster/data/20230117_215100_deposits.pkl')

In [ ]:
import pickle
filename = '/home/fujiju/Documents/GitHub/py0xcluster/data/20230117_215800_all_events.pkl'
fileh = open(filename, 'w')
pickle.dump(uni3_events, fileh)

In [ ]:
uni3_events['swaps'].nunique()